# Example of a Gibson Assembly in pydna
> Visit the full library documentation [here](https://bjornfjohansson.github.io/pydna/)

This example showcases a workflow of modelling Gibson assembly to clone gene fragments into plasmids for synthetic biology. The biological example is sourced [here](https://www.nature.com/articles/s41467-021-24987-9#Sec13). This example constructs a novel synthetic plasmid by joining pyrrolysine aminoacyl tRNA synthetase (PylRS) with a pGEX4T-1 plasmid. As in the biological example, the pydna model below replaced the track of glutathione S-transferase gene and the multiple cloning site sequence, and replaced this track of DNA with the PylRS gene. This places the PylRS gene under the control of Ptac-lacO.  

Source files can be found alongside this notebook, if you would like to follow along. Annotations are made alongside the code to describe key steps.


In [ ]:
# Importing all necessary classes and methods

from pydna.parsers import parse
from pydna.tm import tm_default
from pydna.amplify import pcr
from pydna.dseqrecord import Dseqrecord
from pydna.assembly import Assembly

In [ ]:
# Parsing the files

pGEX4T1_path = "./pGEX-4T-1.gb"
PylRS_path = "./PylRS.gb"
pGEX4T1_docs = parse(pGEX4T1_path)
PylRS_docs = parse(PylRS_path)

# Printing the parsed files

print(pGEX4T1_docs[0].format("gb"))
print(PylRS_docs[0].format("gb"))

LOCUS       pGEX-4T-1               4969 bp    DNA     circular SYN 06-MAR-2024
DEFINITION  Bacterial vector for expressing fusion proteins with a thrombin
            site. For other reading frames, use pGEX-4T-2 or pGEX-4T-3.
ACCESSION   .
VERSION     .
KEYWORDS    .
SOURCE      synthetic DNA construct
  ORGANISM  synthetic DNA construct
            .
REFERENCE   1  (bases 1 to 4969)
  AUTHORS   GE Healthcare
  TITLE     Direct Submission
  JOURNAL   Exported Sep 1, 2024 from SnapGene Viewer 7.2.1
            https://www.snapgene.com
COMMENT     Sequence Label: pGEX-4T-1
FEATURES             Location/Qualifiers
     source          1..4969
                     /lab_host="Escherichia coli"
                     /mol_type="other DNA"
                     /organism="synthetic DNA construct"
     misc_feature    join(183..188,189..204,205..211)
                     /label="tac promoter"
                     /note="strong E. coli promoter; hybrid between the trp and
                     la

In [ ]:
# Isolating the PylRS gene sequence for easy use

PylRS_seq = PylRS_docs[0][2221:3589] # Check the Features list in the GenBank files for the basepair numbers needed to index the sequence.

# Generating primers for the PylRS insert fragment. 

fwd_primer_pGEX = pGEX4T1_docs[0][257:287]
rvs_primer_pGEX_3_start = pGEX4T1_docs[0][950:967]
rvs_primer_pGEX = rvs_primer_pGEX_3_start.reverse_complement() # .reverse_complement() method generates the primer sequence, starting from the 5' end. 

fwd_primer_PylRS = PylRS_seq[0:20]
rvs_primer_PylRS_3_start = PylRS_seq[len(PylRS_seq)-13:]
rvs_primer_PylRS = rvs_primer_PylRS_3_start.reverse_complement()

fwd_primer = fwd_primer_pGEX + fwd_primer_PylRS
rvs_primer = rvs_primer_pGEX + rvs_primer_PylRS

# Generating primers for linearizing the pGEX vector

fwd_primer_pGEX_linear = pGEX4T1_docs[0][966:1010]
rvs_primer_pGEX_linear_3_start = pGEX4T1_docs[0][210:257]
rvs_primer_pGEX_linear = rvs_primer_pGEX_linear_3_start.reverse_complement()

# Showing the sequence of the primers

print(fwd_primer.seq)
print(rvs_primer.seq)
print(fwd_primer_pGEX_linear.seq)
print(rvs_primer_pGEX_linear.seq)

ATGTCCCCTATACTAGGTTATTGGAAAATTATGATGGACAAAAAACCGCT
TGCGGCCGCTCGAGTCGTTACAGGTTGGTA
ATCGTGACTGACTGACGATCTGCCTCGCGCGTTTCGGTGATGAC
GAATACTGTTTCCTGTGTGAAATTGTTATCCGCTCACAATTCCACAC


In [ ]:
# Checking that the primer Tm are matching

print(tm_default(fwd_primer.seq))
print(tm_default(rvs_primer.seq))
print(tm_default(fwd_primer_pGEX_linear.seq))
print(tm_default(rvs_primer_pGEX_linear.seq))


72.03206074687819
76.6435638733937
80.1301376528387
72.60551436106681


In [ ]:
# Performing a PCR to check that the primers are specific. An error message is returned if otherwise.

pcr_product_insert = pcr(fwd_primer, rvs_primer, PylRS_seq)
pcr_product_vector = pcr(fwd_primer_pGEX_linear, rvs_primer_pGEX_linear, pGEX4T1_docs[0])

# Printing out the PCR results

print(pcr_product_insert.format("gb"))
print(pcr_product_vector.format("gb"))

ValueError: PCR not specific! {format(anneal_primers.report()}

In [ ]:
# Performing the Gibson Assembly. Note that the assembly class parameters should be given as a list.

assembled = Assembly([Dseqrecord(pcr_product_insert), Dseqrecord(pcr_product_vector)])
assembled_circ = assembled.assemble_circular()

# Printing out the Gibson Assembly product
print(assembled_circ[0])

IndexError: list index out of range